In [1]:
import pandas as pd
import numpy as np
import os
import json
import torch
from transformers import BertTokenizer
from model import BertForMultiLabelClassification
from multilabel_pipeline import MultiLabelPipeline

In [2]:
# original, group, ekman
taxonomy = 'original'

In [3]:
df_train = pd.read_csv('./data/' + taxonomy + '/' + 'train.tsv', sep= "\t", header=None, converters={1: str})
df_test = pd.read_csv('./data/' + taxonomy + '/' + 'test.tsv', sep= "\t", header=None, converters={1: str})
df_dev = pd.read_csv('./data/' + taxonomy + '/' + 'dev.tsv', sep= "\t", header=None, converters={1: str})

In [4]:
pd.set_option('display.max_colwidth', None)
df_dev

,0,1,2
0,Is this in New Orleans?? I really feel like this is New Orleans.,27,edgurhb
1,"You know the answer man, you are programmed to capture those codes they send you, don’t avoid them!","4,27",ee84bjg
2,I've never been this sad in my life!,25,edcu99z
3,"The economy is heavily controlled and subsidized by the government. In any case, I was poking at the lack of nuance in US politics today","4,27",edc32e2
4,He could have easily taken a real camera from a legitimate source and change the price in Word/Photoshop and then print it out.,20,eepig6r
...,...,...,...
5421,It's pretty dangerous when the state decides which fictional deity is legitimate. For example the [NAME].....,14,edyrazk
5422,I filed for divorce this morning. Hoping he moves out in the next day or so.,20,edi2z3y
5423,"The last time it happened I just said, ""No"" and closed the door",10,eewbqtx
5424,I can’t stand this arrogant prick he’s no better thenFord in Ontario and that guy is a buffoon,3,eefx57m


In [5]:
train_x = df_train[0].to_list()
test_x = df_test[0].to_list()
dev_x = df_dev[0].to_list()

train_y = df_train[1].to_list()
test_y = df_test[1].to_list()
dev_y = df_dev[1].to_list()

In [6]:
model_dir_dict = {'original':'model/bert-base-uncased-goemotions-original-finetuned', 'ekman':'model/bert-base-uncased-goemotions-ekman-finetuned', 'group':'model/bert-base-uncased-goemotions-group-finetuned'}
model_dir = model_dir_dict[taxonomy]

In [7]:
model_dir

'model/bert-base-uncased-goemotions-original-finetuned'

In [8]:
tokenizer = BertTokenizer.from_pretrained(model_dir)
model = BertForMultiLabelClassification.from_pretrained(model_dir)

goemotions = MultiLabelPipeline(
        model=model,
        tokenizer=tokenizer,
        threshold=0.,
        device = 0 if torch.cuda.is_available() else -1
    )

label_list = []
with open(os.path.join("data", taxonomy, "labels.txt"), "r", encoding="utf-8") as f:
    for line in f:
        label_list.append(line.rstrip())

In [9]:
label_list

['admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']

In [10]:
data_x_dict = {'train': train_x, 'test': test_x, 'dev': dev_x}
data_y_dict = {'train': train_y, 'test': test_y, 'dev': dev_y}

data_type = 'test'

data_x = data_x_dict[data_type]
data_y = data_y_dict[data_type]

In [11]:
from sklearn.preprocessing import MultiLabelBinarizer
testing_data_y = []
testing_data_x = []
for idx, i in enumerate(data_y):
    testing_data_x.append(data_x[idx])
    testing_data_y.append(tuple([int(x) for x in i.split(',')]))
multilabelencoder = MultiLabelBinarizer()
testing_data_y = multilabelencoder.fit_transform(testing_data_y)
testing_data_y

array([[0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [12]:
len(testing_data_y)

5427

In [13]:
predicts = []
batch_size = 1000
count_start = 0
count_end = batch_size
data_size = len(testing_data_x)
check = True
while(check):
    if(count_end >= data_size):
        count_end = data_size
        check = False
    print(count_start,count_end)
    testing_data_x_sample = testing_data_x[count_start:count_end]
    testing_data_y_sample = testing_data_y[count_start:count_end]
    predicts += goemotions(testing_data_x_sample)
    count_start += batch_size
    count_end += batch_size

0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 5427


In [14]:
len(predicts)

5427

In [15]:
predicts[0]

{'labels': ['admiration',
  'amusement',
  'anger',
  'annoyance',
  'approval',
  'caring',
  'confusion',
  'curiosity',
  'desire',
  'disappointment',
  'disapproval',
  'disgust',
  'embarrassment',
  'excitement',
  'fear',
  'gratitude',
  'grief',
  'joy',
  'love',
  'nervousness',
  'optimism',
  'pride',
  'realization',
  'relief',
  'remorse',
  'sadness',
  'surprise',
  'neutral'],
 'scores': [0.3023537,
  0.32703906,
  0.31518978,
  0.3240127,
  0.31887832,
  0.39896464,
  0.317414,
  0.30912572,
  0.27703688,
  0.31688893,
  0.32620695,
  0.29099214,
  0.28276277,
  0.30462122,
  0.31252834,
  0.30991644,
  0.3929609,
  0.3185283,
  0.5760638,
  0.2890546,
  0.28986543,
  0.2929844,
  0.2972568,
  0.29906216,
  0.5854968,
  0.6270702,
  0.321214,
  0.40425304]}

In [16]:
th = 0.5
predict_y = []
for predict in predicts:
    ans = np.array(predict['scores'])
    ans[ans > th] = 1
    ans[ans != 1] = 0
    predict_y.append(ans)
predict_y = np.array(predict_y)

In [17]:
pd.DataFrame(goemotions(["Thanks for giving advice to the people who need it! 👌🙏"])[0])

,labels,scores
0,admiration,0.344866
1,amusement,0.303256
2,anger,0.314408
3,annoyance,0.333420
4,approval,0.363553
5,caring,0.334161
6,confusion,0.316989
7,curiosity,0.313076
8,desire,0.307223
9,disappointment,0.319463


In [18]:
testing_data_y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0])

In [19]:
predict_y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0.], dtype=float32)

In [20]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
results = dict()

results["accuracy"] = accuracy_score(testing_data_y, predict_y)
results["macro_precision"], results["macro_recall"], results["macro_f1"], _ = precision_recall_fscore_support(testing_data_y, predict_y, average="macro")
results["micro_precision"], results["micro_recall"], results["micro_f1"], _ = precision_recall_fscore_support(testing_data_y, predict_y, average="micro")
results["weighted_precision"], results["weighted_recall"], results["weighted_f1"], _ = precision_recall_fscore_support(testing_data_y, predict_y, average="weighted")
results["precision"], results["recall"], results["f1"], _ = precision_recall_fscore_support(testing_data_y, predict_y, average=None)

In [21]:
result_list = []
for idx, label in enumerate(label_list):
    result_list.append([label, results['precision'][idx], results['recall'][idx], results['f1'][idx]])
df_result = pd.DataFrame(result_list)
df_result = df_result.sort_values(by=[0])
df_result = df_result.append({0:'macro-average', 1:results['macro_precision'], 2:results['macro_recall'], 3:results['macro_f1']}, ignore_index=True)
df_result.columns = ['Emotion', 'Precision', 'Recall', 'F1']
df_result

,Emotion,Precision,Recall,F1
0,admiration,0.609467,0.817460,0.698305
1,amusement,0.734139,0.920455,0.816807
2,anger,0.434164,0.616162,0.509395
3,annoyance,0.295374,0.518750,0.376417
4,approval,0.333981,0.490028,0.397229
5,caring,0.343590,0.496296,0.406061
6,confusion,0.293413,0.640523,0.402464
7,curiosity,0.434783,0.774648,0.556962
8,desire,0.583333,0.506024,0.541935
9,disappointment,0.262911,0.370861,0.307692


In [22]:
for idx, predict in enumerate(predicts):
    if(len(predict['labels']) >= 1):
        print("=========================Text=========================")
        print(testing_data_x[idx])
        
        print("========================Predice=======================")
        for predict_idx, predict_score in enumerate(predict['scores']):
            if(predict_score > th):  
                print(label_list[int(predict_idx)], predict_score)
                
        print("======================Ground Truth====================")
        for label_idx, label in enumerate(testing_data_y[idx]):
            if(int(label) == 1):
                print(label_list[label_idx])
        print()

=========================Text=========================
I’m really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!
========================Predice=======================
love 0.5760638
remorse 0.5854968
sadness 0.6270702
======================Ground Truth====================
sadness

=========================Text=========================
It's wonderful because it's awful. At not with.
========================Predice=======================
admiration 0.7306483
disgust 0.5294815
======================Ground Truth====================
admiration

=========================Text=========================
Kings fan here, good luck to you guys! Will be an interesting game to watch! 
========================Predice=======================
excitement 0.6086292
optimism 0.70325357
======================Ground Truth====================
excitement

=========================Text=========================
I didn't know that, thank you for teaching me something toda

Thanks a bunch <3
========================Predice=======================
gratitude 0.8250928
======================Ground Truth====================
gratitude

=========================Text=========================
I just want tk be a random scumbag
========================Predice=======================
desire 0.68322873
neutral 0.6270944
======================Ground Truth====================
desire

=========================Text=========================
Very possible but there's absolutely zero indication were working on a deal to trade [NAME] currently. We're not just going to trade him away for peanuts.
========================Predice=======================
disapproval 0.55487657
neutral 0.70857686
======================Ground Truth====================
neutral

=========================Text=========================
I agree that you need more time in the later turns. But most games it feels like people wait until it is their turn to start thinking.
========================Predice=====

Softest fans in sports
========================Predice=======================
neutral 0.71761465
======================Ground Truth====================
neutral

=========================Text=========================
Before the camera was on him, he noticed the atrocious fake jersey on the nimrod beside him. 
========================Predice=======================
neutral 0.79694635
======================Ground Truth====================
neutral

=========================Text=========================
at least it wasn’t the evil [NAME].
========================Predice=======================
relief 0.756377
======================Ground Truth====================
relief

=========================Text=========================
Thanks for having that self awareness, as a physician in training I feel stupid daily. It's a very humbling job
========================Predice=======================
gratitude 0.8240724
======================Ground Truth====================
gratitude

===================

curiosity 0.6468521
neutral 0.59300905
======================Ground Truth====================
curiosity

=========================Text=========================
Thanks Harvard!
========================Predice=======================
gratitude 0.8235481
======================Ground Truth====================
gratitude

=========================Text=========================
What makes it ok as it is?
========================Predice=======================
curiosity 0.6130897
neutral 0.63237685
======================Ground Truth====================
confusion

=========================Text=========================
i am a man and i disagree. i mean you logic is worse if you believe it.
========================Predice=======================
disapproval 0.7789206
======================Ground Truth====================
disapproval

=========================Text=========================
Thanks. And wow! Nice burn.
========================Predice=======================
admiration 0.7264485
gratitude 

=========================Text=========================
Wow. Did u say yes?
========================Predice=======================
curiosity 0.6684508
surprise 0.6673091
======================Ground Truth====================
curiosity
surprise

=========================Text=========================
His returns or his W2?
========================Predice=======================
confusion 0.53291154
curiosity 0.51768583
neutral 0.6941298
======================Ground Truth====================
confusion
neutral

=========================Text=========================
Wouldn't the powder be derived from the crystalline MDMA?
========================Predice=======================
confusion 0.59416586
curiosity 0.65748006
neutral 0.560953
======================Ground Truth====================
neutral

=========================Text=========================
Omg two little blankets for your penis. LOVE THAT.
========================Predice=======================
love 0.8364048
======================

...

======================Ground Truth====================
fear

=========================Text=========================
Bzzzzt, try again
========================Predice=======================
neutral 0.79060316
======================Ground Truth====================
neutral

=========================Text=========================
I own literally no red shirts, would grey, tan, or purplish be okay?
========================Predice=======================
confusion 0.6729262
curiosity 0.6114809
======================Ground Truth====================
neutral

=========================Text=========================
That's a really cheap cop-out for rather common criticism of those games.
========================Predice=======================
neutral 0.76281124
======================Ground Truth====================
annoyance

=========================Text=========================
So can we all collectively accept that [NAME] has a high placement for this week? Okay thank you
========================

approval

=========================Text=========================
You can't post a Sons pic and then say "I got this", that just hurts.
========================Predice=======================
disappointment 0.5648685
sadness 0.68333375
======================Ground Truth====================
disapproval
sadness

=========================Text=========================
Yes - thank you for stating it this way - perfectly said.
========================Predice=======================
gratitude 0.8297918
======================Ground Truth====================
gratitude

=========================Text=========================
Congrats!!! Happy for you.
========================Predice=======================
admiration 0.587547
gratitude 0.71143484
joy 0.64683723
======================Ground Truth====================
joy

=========================Text=========================
Pretty sure the legislature needs to change laws
========================Predice=======================
approval 0.6435679
neutr

disapproval 0.53075606
======================Ground Truth====================
anger

=========================Text=========================
America will lose. You are mentally-deranged and the enemy.
========================Predice=======================
anger 0.5822739
annoyance 0.51643974
======================Ground Truth====================
neutral

=========================Text=========================
As sure as gods got sandals, it sure beats fighting dudes with treasure trails
========================Predice=======================
approval 0.6303042
neutral 0.5641538
======================Ground Truth====================
neutral

=========================Text=========================
Thanks you!
========================Predice=======================
gratitude 0.8390448
======================Ground Truth====================
gratitude

=========================Text=========================
What? Even his follow up comment was related to dog hair.
========================Predice==

======================Ground Truth====================
sadness

=========================Text=========================
Excess of 300 partners and not using protection is unappealing to me, I dont want catch anything 
========================Predice=======================
disapproval 0.7829025
======================Ground Truth====================
disapproval

=========================Text=========================
Seen this result coming. Best result for us and rangers unfortunately.
========================Predice=======================
admiration 0.7803944
======================Ground Truth====================
admiration
approval

=========================Text=========================
just apologize. what other advice could you possibly require? you fucked up, now deal with it.
========================Predice=======================
anger 0.68857425
annoyance 0.506495
remorse 0.51049954
======================Ground Truth====================
anger
remorse

=========================Text=

annoyance 0.5740054
disappointment 0.5231979
sadness 0.55483776
======================Ground Truth====================
annoyance

=========================Text=========================
My kitten just got very happy when Pasta scored. Such loud purrs! I was happy too.
========================Predice=======================
joy 0.8219482
======================Ground Truth====================
joy

=========================Text=========================
The best LGBTQ+ FILM i have seen in years.
========================Predice=======================
admiration 0.8211409
======================Ground Truth====================
admiration

=========================Text=========================
I'm glad you have your kids, but have you seen Brooklyn 99 like the op stated? It's worth living for too.
========================Predice=======================
curiosity 0.61048895
joy 0.6284633
======================Ground Truth====================
admiration
confusion
excitement
joy

===================

In [23]:
# only output the scores that are higher than the threshold
goemotions.threshold = th

In [24]:
goemotions(["I don't believe it is true"], ["I don't believe it is true. It's really good"])

[{'labels': ['disapproval', 'neutral'], 'scores': [0.6078693, 0.50720966]},
 {'labels': ['admiration', 'confusion', 'disapproval'],
  'scores': [0.61421245, 0.5031275, 0.5371183]}]

In [25]:
goemotions(["fucking love you"], ["fuck you"])

[{'labels': ['love'], 'scores': [0.80132246]},
 {'labels': ['anger'], 'scores': [0.7613473]}]